<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-2/Insumos/Acc_Div_SH_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Riesgo de Mercado (Simulación Histórica)

Existen trés métodos principales para medir el riesgo de mercado
* Simulación histórica
* Paramétrico
* Simulación de Monte Carlo



   **Pérdidas y Ganancias simulación histórica (sin alisado)**
   
Un enfoque más claro es suponer que tenemos:
1.   Una matriz $X_{(n+1)\times m}$ de $m$ factores de riesgo y $n+1$ observaciones. 
2.   Denotemos el vector de precios actual como $X_{00}:=(x_{0,1},x_{0,2},\ldots,x_{0,m})$. 
3.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
4.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
5.   Sea $M_{1\times r}=(m_1,\ldots,m_r)$ el vector de posiciones nominales de cada instrumento, es decir, el número de contratos que se tienen por instrumento $m_i\in R$ $(i=1,\ldots,r)$.

In [1]:
!pip install yfinance
import yfinance as yf
import pandas as pd


     |████████████████████████████████| 6.4 MB 17.2 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
start = '2020-01-01'

symbol = ['AAPL','MSFT','SQ','AMZN']
stock_data = yf.download(symbol,start)
# stock_data = stock_data.history(start=start)#, end=end) 
# symbol1='APPL'
# stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


[*********************100%***********************]  4 of 4 completed



##Paso 1.
 (Generación de rendimientos) Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$, es decir
		$${\Delta X}_t=\Big[ T_j(\frac{x_{t,1}}{x_{t+1,1}}), T_j(\frac{x_{t,2}}{x_{t+1,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+1,m}})\Big]\quad t=0,1,\ldots, n-1.$$

In [3]:
# yf.download?
print(stock_data.Close.head(5))

import numpy as np
X=stock_data.Close
X0=stock_data.Close.tail(1)
DeltaX=np.log(stock_data.Close/stock_data.Close.shift(1))

print(DeltaX.head())
print(X0)


                 AAPL         AMZN        MSFT         SQ
Date                                                     
2020-01-02  75.087502  1898.010010  160.619995  63.830002
2020-01-03  74.357498  1874.969971  158.619995  63.000000
2020-01-06  74.949997  1902.880005  159.029999  62.570000
2020-01-07  74.597504  1906.859985  157.580002  64.589996
2020-01-08  75.797501  1891.969971  160.089996  67.599998
                AAPL      AMZN      MSFT        SQ
Date                                              
2020-01-02       NaN       NaN       NaN       NaN
2020-01-03 -0.009770 -0.012213 -0.012530 -0.013089
2020-01-06  0.007937  0.014776  0.002581 -0.006849
2020-01-07 -0.004714  0.002089 -0.009160  0.031774
2020-01-08  0.015958 -0.007839  0.015803  0.045548
                  AAPL         AMZN        MSFT          SQ
Date                                                       
2022-03-28  175.600006  3379.810059  310.700012  138.119995


In [4]:
import pandas as pd
M=np.array([1000,5000,2000,100])
print(M)

#Valoración

Y0=X0.squeeze()*M

Y0



[1000 5000 2000  100]


AAPL    1.756000e+05
AMZN    1.689905e+07
MSFT    6.214000e+05
SQ      1.381200e+04
Name: 2022-03-28 00:00:00, dtype: float64

##Paso 2. 
Construir ${X^\ast}_{n\times m}$ que es la matriz de factores de riesgo simulada basada en el vector de precios actual $X_{00}$
		$$X_{t}^\ast=\Big[x_{0,1}T^{-1}_j(\Delta x_{t,1}), x_{0,2}T^{-1}_j(\Delta x_{t,2}),\ldots, x_{0,m}T^{-1}_j(\Delta x_{t,m})\Big]\quad (t=1,\ldots,n\quad j=1,2),$$
		donde $T_1(x)=\ln(x)$, $T_2(x)=x-1$.



In [5]:
X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
X_s.head

<bound method NDFrame.head of                   AAPL         AMZN        MSFT          SQ
Date                                                       
2020-01-02         NaN          NaN         NaN         NaN
2020-01-03  173.892817  3338.782374  306.831253  136.323977
2020-01-06  176.999230  3430.120525  311.503115  137.177271
2020-01-07  174.774151  3386.879121  307.867125  142.579032
2020-01-08  178.424758  3353.418283  315.648961  144.556618
...                ...          ...         ...         ...
2022-03-22  179.252591  3450.915325  315.789022  145.260585
2022-03-23  177.045829  3349.453161  306.030206  131.895390
2022-03-24  179.582240  3384.805147  315.482583  137.773169
2022-03-25  176.255713  3403.023715  310.270883  133.282834
2022-03-28  176.484443  3466.308640  317.882310  146.375606

[564 rows x 4 columns]>

## Paso 3. 
Construcción de la matriz de reevaluación del portafolio basada en los escenarios históricos $Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$Y_t=[m_1f_1(X_{t}^\ast), m_2f_2(X_{t}^\ast),\ldots, m_rf_r(X_{t}^\ast)]\quad (t=1,\ldots,n).$$
    


In [6]:
Y_s=X_s*M.squeeze()
Y_s.head


<bound method NDFrame.head of                      AAPL          AMZN           MSFT            SQ
Date                                                                
2020-01-02            NaN           NaN            NaN           NaN
2020-01-03  173892.816607  1.669391e+07  613662.506754  13632.397686
2020-01-06  176999.230013  1.715060e+07  623006.229770  13717.727067
2020-01-07  174774.151191  1.693440e+07  615734.249743  14257.903184
2020-01-08  178424.758422  1.676709e+07  631297.921544  14455.661849
...                   ...           ...            ...           ...
2022-03-22  179252.590647  1.725458e+07  631578.043834  14526.058535
2022-03-23  177045.829411  1.674727e+07  612060.411556  13189.539023
2022-03-24  179582.240426  1.692403e+07  630965.165377  13777.316872
2022-03-25  176255.713160  1.701512e+07  620541.766114  13328.283387
2022-03-28  176484.443270  1.733154e+07  635764.620085  14637.560641

[564 rows x 4 columns]>

## Paso 4. 
Construcción de la matriz de pérdidas y ganancias del portafolio basada en los escenarios históricos $\Delta Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$\Delta Y_t=Y_t-Y_{0}=[m_1(f_1(X_{00})-f_1(X_{t}^\ast)), m_2( f_2(X_{00})-f_2(X_{t}^\ast)),\ldots, m_r(f_r(X_{00})-f_r(X_{t}^\ast))],$$
		para $(t=1,\ldots,n)$. Se puede obtener el vector de pérdidas totales ${\Delta YT}_{n\times m}$ muy fácilmente, esto es $\Delta YT=\sum_{k=1}^r Y_{kt}$ $(t=1,\ldots,n)$, incluso se puede hacer lo mismo por tipo de riesgo.


In [7]:
PL=Y_s-Y0

cols=PL.shape[1]

PL['Total']=PL.sum(axis=1)
PL.head

<bound method NDFrame.head of                    AAPL           AMZN          MSFT          SQ  \
Date                                                               
2020-01-02          NaN            NaN           NaN         NaN   
2020-01-03 -1707.189497 -205138.422277  -7737.517660 -179.601826   
2020-01-06  1399.223909  251552.333451   1606.205355  -94.272445   
2020-01-07  -825.854913   35345.313385  -5665.774671  445.903672   
2020-01-08  2824.752319 -131958.879173   9897.897130  643.662338   
...                 ...            ...           ...         ...   
2022-03-22  3652.584544  355526.332495  10178.019420  714.059023   
2022-03-23  1445.823307 -151784.487015  -9339.612858 -622.460488   
2022-03-24  3982.234322   24975.440416   9565.140963  -34.682640   
2022-03-25   655.707057  116068.280581   -858.258300 -483.716125   
2022-03-28   884.437166  432492.906823  14364.595671  825.561129   

                    Total  
Date                       
2020-01-02       0.000000  
2

##Paso 5. 
Obtenerla medida de riesgo basado en un nivel de confianza de la matriz ${\Delta Y}_{\cdot k}$ $(k=1,\ldots,r)$ y $\Delta YT$.

Las pérdidas y ganancias vía simulación histórica sin alisado es donde se ponderan los escenarios históricos con un mismo peso para todos, es decir cada escenario tiene una probabilidad de ocurrencia de $1/n$.

In [8]:
alpha = 0.98 #@param {type:"slider", min:0.8, max:1, step:0.001}

###Valor en Riesgo (VaR)

Es un cuantil a un $\alpha$ de confianza de la distribución de pérdidas y ganancias, generalmente se expresa en términos monetarios.

#VaR

El valor en riesgo (VaR) es una estadística utilizada para tratar de cuantificar el nivel de riesgo financiero dentro de una empresa o cartera durante un período de tiempo específico. VaR proporciona una estimación de la pérdida máxima de una posición o cartera determinada durante un período de tiempo, y puede calcularla a través de varios niveles de confianza.

Estimar el riesgo de una cartera es importante para el crecimiento del capital a largo plazo y la gestión del riesgo, en particular dentro de empresas o instituciones más grandes. El VaR generalmente se enmarca como algo como esto:

“Tenemos un VaR de cartera de 250.000 USD para el próximo mes al 95% de confianza”
Esto significa que, con un 95% de confianza, podemos decir que la pérdida de la cartera no superará los 250.000 USD en un mes.
En esta publicación, lo guiaré a través de los pasos para calcular esta métrica en una cartera de acciones.

In [30]:
VaR=PL.quantile(1-alpha, numeric_only=False)

print(VaR)


VaRsi=[]

for i in range(0,PL.shape[1]):
  VC=PL.iloc[:,i].sort_values(ascending=True).quantile(1-alpha, interpolation = 'higher')
  VaRsi.append(VC)

print(VaRsi)

print(sum(Y0))

AAPL      -8098.885033
AMZN    -772667.836840
MSFT     -26673.716986
SQ        -1097.098274
Total   -823385.570222
Name: 0.020000000000000018, dtype: float64
[-7992.68713376089, -742618.241096029, -26299.24082173768, -1037.2854851955035, -788737.1589506161]
17709862.322998047


###Valor en Riesgo Condicional o "Expected Shortfall"

Es el promedio de las pérdidas que son más severas que el Valor en Riesgo ($VaR_\alpha$) a un $\alpha$ de confianza, es decir:
$$E(x|x\le VaR_\alpha)$$


In [ ]:
def VaR_Cond(x , VaR ):
  return np.mean(x[x <= VaR])

In [ ]:

VaR_C=[]

for i in range(0,PL.shape[1]):
  VC=VaR_Cond(PL.iloc[:,i],VaR[i])
  VaR_C.append(VC)


In [ ]:

print(VaR)
print(VaR_C)

pd.concat([VaR,pd.DataFrame(VaR_C).squeeze()], axis=1, ignore_index=True)

AAPL      -7298.874401
AMZN    -630538.408388
MSFT     -23795.190363
SQ         -813.530388
Total   -677462.575261
Name: 0.020000000000000018, dtype: float64
[-11261.958141897987, -846332.0609951812, -36937.994447413366, -1200.8464660778375, -881792.5615757715]


,0,1
AAPL,-7298.874401,NaN
AMZN,-630538.408388,NaN
MSFT,-23795.190363,NaN
SQ,-813.530388,NaN
Total,-677462.575261,NaN
0,NaN,-11261.958142
1,NaN,-846332.060995
2,NaN,-36937.994447
3,NaN,-1200.846466
4,NaN,-881792.561576
